In [2]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import gym
import matplotlib.pyplot as plt
%matplotlib inline

try:
    xrange = xrange
except:
    xrange = range

In [3]:
env = gym.make('CartPole-v0')

In [4]:
gamma = 0.99

def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r) #DICOUNT_SIZE=R_SIZE
    running_add = 0 #INITIALIZE
    for t in reversed(xrange(0, r.size)):
        running_add = running_add * gamma + r[t] #new_reward=reward+gamma*previous_reward
        discounted_r[t] = running_add
    return discounted_r

In [5]:
class agent():
    def __init__(self, lr, s_size,a_size,h_size): #_,State_size,action_size,hidden layer_size
        #These lines established the feed-forward part of the network. The agent takes a state and produces an action.
        self.state_in= tf.placeholder(shape=[None,s_size],dtype=tf.float32) # Placeholder to hold a float32 vector of size sstate size
        hidden = slim.fully_connected(self.state_in,h_size,biases_initializer=None,activation_fn=tf.nn.relu)
        self.output = slim.fully_connected(hidden,a_size,activation_fn=tf.nn.softmax,biases_initializer=None)
        self.chosen_action = tf.argmax(self.output,1)
        
        #NN is [s_size->(nn.relu) hidden_layer_size->(nn.softmax) action_size-> max_probablity]
        
        #The next six lines establish the training proceedure. We feed the reward and chosen action into the network
        #to compute the loss, and use it to update the network.
        #Placeholders for reward and action
        self.reward_holder = tf.placeholder(shape=[None],dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[None],dtype=tf.int32)
        
        #Creates an increasing array of [0,1,.. size of(output) in 0th axis] * size  of (output) in 1st axis + action_holder
        self.indexes = (tf.range(0, tf.shape(self.output)[0]) * (tf.shape(self.output)[1])) + self.action_holder 
        #Gathers the important outputs, reshapes output to 1 hot vector, and select important elements by by self.indexes
        self.responsible_outputs = tf.gather(tf.reshape(self.output, [-1]), self.indexes)  

        # Loss = mean(log(outputs)*rewards) #Reduce mean computes mean of given set of vecotrs
        self.loss = -tf.reduce_mean(tf.log(self.responsible_outputs)*self.reward_holder)
        
        tvars = tf.trainable_variables()
        #Gradient Holder
        self.gradient_holders = []
        #http://book.pythontips.com/en/latest/enumerate.html
        for idx,var in enumerate(tvars):
            #Placeholder for every trainable variable
            placeholder = tf.placeholder(tf.float32,name=str(idx)+'_holder')
            self.gradient_holders.append(placeholder)
        #Computes gradient of loss wrt tvars
        self.gradients = tf.gradients(self.loss,tvars)
        
        optimizer = tf.train.AdamOptimizer(learning_rate=lr)
        #https://www.programiz.com/python-programming/methods/built-in/zip
        self.update_batch = optimizer.apply_gradients(zip(self.gradient_holders,tvars))

In [6]:
tf.reset_default_graph() #Clear the Tensorflow graph.

myAgent = agent(lr=1e-2,s_size=4,a_size=2,h_size=8) #Load the agent.

total_episodes = 1000 #Set total number of episodes to train agent on.
max_ep = 999
update_frequency = 5

init = tf.global_variables_initializer()

# Launch the tensorflow graph
with tf.Session() as sess:
    sess.run(init) #start
    i = 0
    total_reward = []
    total_length = []
        
    gradBuffer = sess.run(tf.trainable_variables())

    for ix,grad in enumerate(gradBuffer):
        gradBuffer[ix] = grad * 0 #Gradient initialize to zero.

    while i < total_episodes:
        s = env.reset() #Reset the enviroment ange get the states
        running_reward = 0
        ep_history = []
        for j in range(max_ep):
            #Probabilistically pick an action given our network outputs.
            a_dist = sess.run(myAgent.output,feed_dict={myAgent.state_in:[s]}) #Run the neural net with input state as s
            a = np.random.choice(a_dist[0],p=a_dist[0])# Make a Random choice between element of a_dist with probabilities of that element
            a = np.argmax(a_dist == a)#Check the index of the random choice
            
            s1,r,done,_ = env.step(a) #Get our reward for taking an action given a bandit.
            ep_history.append([s,a,r,s1]) #Append the episode history
            s = s1
            running_reward += r #r is reward
            if done == True:
                #Update the network.
                #Convert into numpy array
                ep_history = np.array(ep_history)
                #Calculate discount rewards based on walue of r
                ep_history[:,2] = discount_rewards(ep_history[:,2])
                #Feed Dictionary consisting of episode history[reward] -> reward holder, episode history[action] -> actionholder and episode history[state]->state
                feed_dict={myAgent.reward_holder:ep_history[:,2],
                        myAgent.action_holder:ep_history[:,1],myAgent.state_in:np.vstack(ep_history[:,0])}
                grads = sess.run(myAgent.gradients, feed_dict=feed_dict) #Compute gradient
                for idx,grad in enumerate(grads):
                    gradBuffer[idx] += grad

                if i % update_frequency == 0 and i != 0:
                    feed_dict= dictionary = dict(zip(myAgent.gradient_holders, gradBuffer))
                    _ = sess.run(myAgent.update_batch, feed_dict=feed_dict)
                    for ix,grad in enumerate(gradBuffer):
                        gradBuffer[ix] = grad * 0
                
                total_reward.append(running_reward)
                total_length.append(j)
                break

        
            #Update our running tally of scores.
        if i % 100 == 0:
            print(np.mean(total_reward[-100:]))
        i += 1

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/home/yukrav/.conda/envs/RL/lib/python3.7/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


33.0
26.89
39.06
49.17
59.11


KeyboardInterrupt: 